In [1]:
# --- Core Libraries ---
import os
import random
import json
import pandas as pd
import numpy as np

# --- Hugging Face: Dataset, Tokenizer, Model ---
from datasets import load_dataset, DatasetDict
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    TrainingArguments, 
    Trainer, 
    DataCollatorForLanguageModeling, 
    pipeline
)

# --- LoRA & Parameter-Efficient Tuning ---
from peft import LoraConfig, get_peft_model, TaskType

# --- W&B Experiment Tracking ---
import wandb

# --- SQL Evaluation ---
import sqlite3
import sqlparse
from tabulate import tabulate
import evaluate  # for BLEU, ROUGE

In [2]:
import torch

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    print("GPU not detected — will fall back to CPU.")

PyTorch version: 2.5.1+cu121
CUDA available: True
Using GPU: NVIDIA GeForce RTX 4050 Laptop GPU


In [4]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("Clinton/Text-to-SQL-v1")

#Explore Sample Rows
import pandas as pd

df = pd.DataFrame(dataset["train"])
df.sample(5)

,instruction,input,response,source,text
73589,Where is the home venue of Bengaluru FC?,"CREATE TABLE table_66768 (\n ""Team"" text,\n...","SELECT ""Home venue"" FROM table_66768 WHERE ""Te...",wikisql,Below are sql tables schemas paired with instr...
80112,from PHOENIX to LAS VEGAS on saturday,CREATE TABLE code_description (\n code varc...,SELECT DISTINCT flight.flight_id FROM airport_...,atis,Below are sql tables schemas paired with instr...
60315,What's the Loss listed with a Score of 5-1?,CREATE TABLE table_name_60 (\n loss VARCHAR...,"SELECT loss FROM table_name_60 WHERE score = ""...",sql_create_context,Below are sql tables schemas paired with instr...
119558,What is the away side that playes at lake oval?,CREATE TABLE table_name_12 (\n away_team VA...,SELECT away_team AS score FROM table_name_12 W...,sql_create_context,Below are sql tables schemas paired with instr...
35749,In what venue was the event held on 14 Februar...,CREATE TABLE table_name_95 (\n venue VARCHA...,"SELECT venue FROM table_name_95 WHERE date = ""...",sql_create_context,Below are sql tables schemas paired with instr...
